In [ ]:
#!cat /proc/cpuinfo

In [1]:
# connected GPU information
!nvidia-smi

Sun Feb 14 06:30:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [5]:
# install huggingface transformers
!pip -q install transformers

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 890kB 36.7MB/s 
     |████████████████████████████████| 3.2MB 47.4MB/s 


In [6]:
import os

import tensorflow as tf
import transformers

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random as rnd

In [7]:
import re
import string

import nltk

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
import warnings
warnings.filterwarnings('ignore')

<a name='1'></a>
#Importing the Data

In [9]:
data = pd.read_csv("/content/drive/My Drive/Duplicate Question Detection/quora_questions.csv")
#data.fillna("none value", inplace=True) # replace nan value to none
# drop the rows with null value
data.dropna(axis=0, inplace=True)
N=len(data)
print('Number of question pairs: ', N)
data.head()

Number of question pairs:  404348


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
# def cut_to_max(text, max_len):
#     words = text.split()[:max_len]
#     return ' '.join(words)

# # test the func
# # cut_to_max(' abc test wewe dfd ddd', 3)

In [10]:
def cleaning_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    #text = text.lower()

    # Clean the text
    #text = re.sub(r"[^A-Za-z0-9^,!.?\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"What's", "What is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"Can't", "Cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"I'm", "I am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " , ", text)
    text = re.sub(r"\.", " . ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text

In [11]:
data['question1'] = data['question1'].apply(lambda x: cleaning_text(x))
data['question2'] = data['question2'].apply(lambda x: cleaning_text(x))

In [12]:
# for s in data['question1']:
#     if len(s.split()) > 63:
#         print(s)


In [13]:
print("dataset labels Distribution")
print(data.is_duplicate.value_counts())

dataset labels Distribution
0    255042
1    149306
Name: is_duplicate, dtype: int64


We first split the data into a train and test set. The test set will be used later to evaluate our model.

In [14]:
# model selection
from sklearn.model_selection import train_test_split
data_train , data_test = train_test_split(data, train_size=0.75, random_state=0)
# validation_data, _ = train_test_split(data_train, train_size=0.1, random_state=0)
len(data_train), len(data_test)

(303261, 101087)

In [ ]:
# print("Train dataset Distribution")
# print(data_train.is_duplicate.value_counts())

# print("\n\nTest dataset Distribution")
# print(data_test.is_duplicate.value_counts())

In [15]:
train_Q1 = np.array(data_train['question1'])
train_Q2 = np.array(data_train['question2'])
t_labels = np.array(data_train['is_duplicate'])
# make it one-hot encoding
train_labels = tf.keras.utils.to_categorical(t_labels, num_classes=2)

test_Q1 = np.array(data_test['question1'])
test_Q2 = np.array(data_test['question2'])
te_labels  = np.array(data_test['is_duplicate'])
# make it one-hot encoding
test_labels = tf.keras.utils.to_categorical(te_labels, num_classes=2)

In [ ]:
train_Q1.shape, train_Q2.shape, train_labels.shape, test_Q1.shape

((303261,), (303261,), (303261, 2), (101087,))

In [16]:
# data generator
class data_generator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [17]:
# make a model for detecting duplicate question
# max_length = 256
# learning_rate = 0.001
def duplicate_question_detection_model():

    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )

    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )

    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )

     # Loading pretrained BERT model.
    bert_model = transformers.TFRobertaModel.from_pretrained('roberta-base')

    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = True

    outputs = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )

    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(outputs.last_hidden_state)

    # flat = tf.keras.layers.Flatten()(bi_lstm)

    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)

    # dropout = tf.keras.layers.Dropout(rate=0.1)(pooled_output)
    # output = tf.keras.layers.Dense(2, activation="softmax")(dropout)

    # model object
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )

    return model


In [18]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    max_length=64
    learning_rate=5e-5
    model = duplicate_question_detection_model()

# print(f"Strategy: {strategy}")
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f1a104022a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f1a104022a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf

In [19]:
# data preparation
batch_size = 64
train_data = data_generator(
    data_train[["question1", "question2"]].values.astype("str"),
    train_labels,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = data_generator(
    data_test[["question1", "question2"]].values.astype("str"),
    test_labels,
    batch_size=batch_size,
    shuffle=False,
)

In [21]:
# change learning rate

tf.keras.backend.set_value(model.optimizer.lr, 1e-5)
print(tf.keras.backend.get_value(model.optimizer.lr))

1e-05


In [22]:
epochs = 1
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
4738/4738 [==============================] - ETA: 0s - loss: 0.1620 - acc: 0.9369

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


4738/4738 [==============================] - 2406s 503ms/step - loss: 0.1620 - acc: 0.9369 - val_loss: 0.2440 - val_acc: 0.9025


In [ ]:
model.save_weights('/content/drive/My Drive/Duplicate Question Detection/roberta_bi_lstm_pooling_4_epoch_random_0_cleaned_text.h5', overwrite=True, save_format=None, options=None)

In [20]:
model.load_weights('/content/drive/My Drive/Duplicate Question Detection/roberta_bi_lstm_pooling_4_epoch_random_0_cleaned_text.h5', by_name=False, skip_mismatch=False, options=None)

In [ ]:
model.evaluate(valid_data, verbose=1)

In [ ]:
preds = model.predict(
    valid_data,
    batch_size=batch_size,
    use_multiprocessing=True
)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix
y_pred = np.argmax(preds, axis=1)
n = y_pred.shape[0]
y_true = te_labels[:n]
print('accuracy: ', accuracy_score(y_true, y_pred))
print('F1 score: ', f1_score(y_true, y_pred))
print('Recall score: ', recall_score(y_true, y_pred))
print('confusion matrix: \n', confusion_matrix(y_true, y_pred))

In [ ]:
# draw curves 
import matplotlib.pyplot as plt
%pylab inline
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='val')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()